In [502]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from netgen.csg import Pnt


In [503]:
#help(unit_square)
l = 0

mw = 1.0
for i in range(l):
    mw = mw/2


netgen_mesh = unit_square.GenerateMesh(maxh=mw, segmentsperedge=l+1.2, grading=0.1)

mesh = Mesh(netgen_mesh)
#help(netgen_mesh)
unref_verts = mesh.vertices

unref_edges = mesh.edges

nedges = mesh.nedge

parent_el = mesh.ne
#issues with this-> maskes certain spaces not work
# mesh.Refine()
# mesh.Refine()
# mesh.Refine()
#other spaces dont work
H_h = HDiv(mesh,order=0,RT=True) #order_policy=ORDER_POLICY.CONSTANT
#might need to be number of faces (in unrefined mesh) because elements are have 1 face
unref_dofnr = H_h.ndof #mesh.ne
print(unref_dofnr)
print(mw)
elvol = Integrate(CoefficientFunction(1),mesh,element_wise=True)
mesh_h = [(2*vol)**(1/2) for vol in elvol]
print("actual meshwidth range",min(mesh_h),max(mesh_h), "\n")

Draw(mesh)

help(netgen_mesh)

5
1.0
actual meshwidth range 0.999999999999998 0.9999999999999981 



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Help on Mesh in module netgen.libngpy._meshing object:

class Mesh(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      Mesh
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Add(...)
 |      Add(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. Add(self: netgen.libngpy._meshing.Mesh, arg0: netgen.libngpy._meshing.MeshPoint) -> netgen.libngpy._meshing.PointId
 |      
 |      2. Add(self: netgen.libngpy._meshing.Mesh, arg0: netgen.libngpy._meshing.Element3D) -> netgen.libngpy._meshing.ElementId3D
 |      
 |      3. Add(self: netgen.libngpy._meshing.Mesh, arg0: netgen.libngpy._meshing.Element2D) -> netgen.libngpy._meshing.ElementId2D
 |      
 |      4. Add(self: netgen.libngpy._meshing.Mesh, el: netgen.libngpy._meshing.Element1D, project_geominfo: bool = False) -> netgen.libngpy._meshing.ElementId1D
 |      
 |      5. Add(self: netgen.libngpy._meshing.Mesh, arg0: netgen.libngpy._meshing.Element

In [504]:
#Important

elnrs = np.ones((nedges,2),dtype=int)

for e in mesh.edges:
    el1 = mesh.__getitem__(e.elements[0])
    elnrs[e.nr,:] = el1.nr
    if len(e.elements) == 2:
        el2 = mesh.__getitem__(e.elements[1])
        elnrs[e.nr,1] = el2.nr

print(elnrs)

[[0 0]
 [0 1]
 [1 1]
 [0 0]
 [1 1]]


In [505]:


mesh.ngmesh.SplitPowellSabin()
mesh.ngmesh.Update()

#mesh._updateBuffers()

H_h.Update() 

H_h = Compress(H_h)


#might need to be number of faces in barycentrically refined mesh
bary_dofnr = H_h.ndof #mesh.ne
print(bary_dofnr)

#mesh._updateBuffers()


Draw(mesh)

22


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [506]:
data_length = 24 * nedges

data = np.zeros(data_length)
row_ind = np.zeros(data_length, dtype=int)
col_ind = np.zeros(data_length, dtype=int)

j = 0

for e in unref_edges:
    parent = e.nr  
    #print(parent)
    v0 = e.vertices[0]  
    v1 = e.vertices[1] 
    v0 = mesh.__getitem__(v0)
    v1 = mesh.__getitem__(v1)



    for el in v0.elements:
        if mesh.ngmesh.MacroElementNr(el.nr) == elnrs[parent,0] or mesh.ngmesh.MacroElementNr(el.nr) == elnrs[parent,1]:
            el = mesh.__getitem__(el)
            for child in el.edges:
                
                child = mesh.__getitem__(child)
                if len(child.elements) == 2:
                    
                    par_el1 = mesh.ngmesh.MacroElementNr(child.elements[0].nr)
                    par_el2 = mesh.ngmesh.MacroElementNr(child.elements[1].nr)

                    #same parent element
                    if  par_el1 == par_el2:
                        row_ind[j] = parent
                        col_ind[j] = H_h.GetDofNrs(child)[0]
                        data[j] = 1/6 
                        j += 1
                    #different parent, but both are elements of parent edge (the children of the original edge fulfill this)
                    elif par_el1 != par_el2 and (par_el1 == elnrs[parent,0] or par_el1 == elnrs[parent,1]) and (par_el2 == elnrs[parent,0] or par_el2 == elnrs[parent,1]):
                        row_ind[j] = parent
                        col_ind[j] = H_h.GetDofNrs(child)[0]
                        data[j] = 3/12 
                        j += 1
                #boundry
                elif len(child.elements) == 1:
                    #the edge midpoint always has higher number than the coarse vertices, s it will always be at index 1
                    v2 = child.vertices[1]
                    v2 = mesh.__getitem__(v2)
                    if(set(v2.edges).intersection(v1.edges) != set()):
                        row_ind[j] = parent
                        col_ind[j] = H_h.GetDofNrs(child)[0]
                        data[j] = 3/6 
                        j += 1                    



    #print("\n")

    for el in v1.elements:
        if mesh.ngmesh.MacroElementNr(el.nr) == elnrs[parent,0] or mesh.ngmesh.MacroElementNr(el.nr) == elnrs[parent,1]:
            el = mesh.__getitem__(el)
            for child in el.edges:
                
                child = mesh.__getitem__(child)
                if len(child.elements) == 2:
                    
                    par_el1 = mesh.ngmesh.MacroElementNr(child.elements[0].nr)
                    par_el2 = mesh.ngmesh.MacroElementNr(child.elements[1].nr)

                    #same parent element
                    if  par_el1 == par_el2:
                        row_ind[j] = parent
                        col_ind[j] = H_h.GetDofNrs(child)[0]
                        data[j] = -1/6 
                        j += 1
                    #different parent, but both are elements of parent edge (the children of the original edge fulfill this)
                    elif par_el1 != par_el2 and (par_el1 == elnrs[parent,0] or par_el1 == elnrs[parent,1]) and (par_el2 == elnrs[parent,0] or par_el2 == elnrs[parent,1]):
                        row_ind[j] = parent
                        col_ind[j] = H_h.GetDofNrs(child)[0]
                        data[j] = -3/12 
                        j += 1
                #boundry
                elif len(child.elements) == 1:
                    #the edge midpoint always has higher number than the coarse vertices, so it will always be at index 1
                    v2 = child.vertices[1]
                    v2 = mesh.__getitem__(v2)
                    if(set(v2.edges).intersection(v0.edges) != set()):
                        row_ind[j] = parent
                        col_ind[j] = H_h.GetDofNrs(child)[0]
                        data[j] = -3/6 
                        j += 1     




ng_mat = la.SparseMatrixdouble.CreateFromCOO(row_ind,col_ind,data, nedges,mesh.nedge - nedges)
print(ng_mat.ToDense())

     0.5       0 0.333333       0       0    -0.5       0 -0.333333       0       0       0       0       0       0       0       0       0 -0.166667 0.166667       0       0       0
       0     0.5 0.333333       0 0.333333       0       0       0       0    -0.5 -0.333333       0 -0.333333       0       0       0 0.166667 -0.166667       0       0 -0.166667 0.166667
       0       0       0     0.5 0.333333       0       0       0       0       0       0       0       0       0    -0.5 -0.333333       0       0       0 0.166667 -0.166667       0
       0       0       0       0       0       0     0.5 0.333333    -0.5       0 -0.333333       0       0       0       0       0 0.166667       0 -0.166667       0       0       0
       0       0       0       0       0       0       0       0       0       0       0     0.5 0.333333    -0.5       0 -0.333333       0       0       0 0.166667       0 -0.166667



In [507]:
dfu = GridFunction(H_h,multidim = unref_dofnr)
#help(ng_mat)



dfu.Set((0,0))
for i in range(unref_dofnr):
    #print(ng_mat.ToDense().NumPy()[:,i])
    dfu.vecs[:][i] = ng_mat.ToDense().NumPy()[i,:]


# for i in range(4):
    #print(dfu.vecs[i][:].FV().NumPy())

Draw(dfu, animate = False, min = 0, max = 2, deformation = False)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [508]:
#gfu.Set(0) 

# for e in mesh.edges:
#     print(e.nr)
#     dofnr = H_h.GetDofNrs(e)[0]
#     print(dofnr)

# gfu = GridFunction(H_h,multidim = H_h.ndof, autoupdate=False)

# for i in range(H_h.ndof):
#     gfu.vecs[i][i] = 3/6


# gfu.vecs[0][5] = -3/6

# gfu.vecs[0][2] = 2/6

# gfu.vecs[0][7] = -2/6

# gfu.vecs[0][17] = -1/6

# gfu.vecs[0][18] = 1/6



# Draw(gfu, animate = False, min = 0, max = 2.0, deformation = False)

In [509]:
netgen_mesh = unit_square.GenerateMesh(maxh=mw, segmentsperedge=l+1.2, grading=0.1)

ref_mesh = Mesh(netgen_mesh)

H_h = HDiv(ref_mesh,order=0,RT=True)

gfu = GridFunction(H_h,multidim = H_h.ndof)

#gfu.Set(0) 

for i in range(H_h.ndof):
    gfu.vecs[i][i] = 1.0

Draw(gfu, animate = False, min = 0, max = 2.0, deformation = False)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [510]:
netgen_mesh = unit_square.GenerateMesh(maxh=1)

netgen_mesh.SplitPowellSabin()

ref_mesh = Mesh(netgen_mesh)

H_h = HDiv(ref_mesh,order=0,RT=True)

gfu = GridFunction(H_h,multidim = H_h.ndof)

#gfu.Set(0) 

for i in range(H_h.ndof):
    gfu.vecs[i][i] = 1.0

Draw(gfu, animate = True, min = 0, max = 1.0, deformation = False)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene